<a href="https://colab.research.google.com/github/justcme/PerformanceSpecs/blob/main/I_NewCriteriaAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Lipid Panel Assays Performance Recommendations

Aim to add known biological variability, then NEW allowed imprecision and bias (as proportional bias) to given "true" values, to determine the effect on variability of risk classification.

# SETUP

OS

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive/Colab Notebooks/LipidPerf/from_Pandas")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Imports

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import random
from tqdm import tqdm

# DATA

In [ ]:
df_base = pd.read_csv("Base/df_base_new.csv")  #Original Lipid Data
PCE_coeffs = pd.read_csv("Base/PCE_coeffs.csv")   #Coefficients for Pooled Cohort Equation calculation
TAE = pd.read_csv('Base/TAEnew_df.csv', index_col = 0)   #Allowable error and BV lookup table
df0 = pd.read_csv('Base/df0.csv')

In [ ]:
TAE

,BV,PB,CV,SD
Parameter,,,,
TC,0.053,0.02,0.03,NaN
HDL,0.058,0.04,0.03,1.7
TG,0.200,0.04,0.03,NaN
dLDL,0.083,0.04,0.04,NaN


#Create Global Variables

In [ ]:
params = ['TC','HDL','TG','dLDL']
BLipTypes = ['cLDL','NHDL','dLDL']

#PCE Variables
age = df_base['age']
SBP = df_base['SBP']
BPRx = df_base['BPRx']
BPRxn = df_base['BPRxn']
smoker = df_base['smoker']
DM = df_base['DM']

# FUNCTIONS

## 1. Add Variability

In [ ]:
Seeds2 = [334, 376,  81, 213, 163, 441, 417, 332,  24, 445, 295, 420, 461,
       446, 185, 472, 430,  92,  99, 250, 136,  94, 337, 332,  24, 174,
       324, 479, 322, 456,  31, 279, 379, 123, 262, 394,  75, 169,  76,
       133, 339, 471, 304, 285, 339, 481, 227,  97, 294, 425, 189, 168,
       227,  38, 160, 467,  14, 116,  99, 474,  89, 282, 158,  33, 330,
       332, 493,  81, 218, 326, 341, 378,  64, 176,  67, 340, 365, 405,
       129, 237, 488, 203, 264, 471,  76, 110, 180, 385,  81, 157, 233,
       110, 236, 116, 134,  97, 144, 154, 497, 230,  34, 296,  98, 350,
       443,  63, 244, 350, 229, 293, 473, 484, 322, 411, 304, 186,  24,
       291, 144, 191, 396, 421, 199, 328, 263, 317,  11, 363, 106, 437,
       276, 182, 256, 406, 379,  51, 313,  13,  79, 293, 311, 162, 214,
       312, 466, 105, 336, 421, 255, 426,  40, 469, 381, 118, 466, 131,
       452, 411, 408, 491, 398,  33, 170, 407, 130,  18,   3, 225,   3,
       422, 114,  42, 426, 498, 473,  14, 456, 179, 175,  84,  77,  48,
        97, 295, 455, 176,  12, 100, 198, 379,  78, 383, 178, 263,  81,
       488, 411, 343, 204, 419,  18, 187, 342, 414, 246, 353, 160, 317,
       411, 473, 232, 250, 277, 284, 127, 162, 493, 179, 339,  97, 295,
       259, 432,  86, 154,  12, 476,   9, 352,  37, 402, 295, 364,  71,
       200, 384, 461, 305, 107, 315, 240,  88, 371, 210, 328, 466, 361,
       378,  85, 480, 150, 267, 275, 289, 256,  93, 272, 167,  41,  80,
       118, 308, 410, 137, 201, 292, 366, 310, 208,   3,  73,  27,  67,
       235, 328, 254, 482, 221,  39, 174,  70, 248, 398, 150,  78, 295,
       324, 476, 299, 107, 364, 154, 335, 345, 106, 472, 255, 153, 484,
       402,  96, 354, 183, 186, 320, 472, 235, 499, 244,  96, 435, 400,
       202, 455, 186, 410, 447, 475, 141, 486, 274,  25, 318, 224, 436,
         5,  43, 230, 246, 270,  12, 444, 454,  58, 164, 476,  78, 368,
        31, 489, 189, 487, 306, 257,  61,  59,  38, 276, 184, 332, 185,
        14,  36, 247, 344, 303,  90, 341, 455,  94, 294, 429, 147, 141,
       176, 334, 407, 463, 242, 390,  42, 122, 196, 202, 240, 214,  52,
        45, 167, 426, 164,  37,  36, 292, 216, 148, 460,  25, 400, 200,
       399, 157,  77,  18, 142,  86, 101, 226, 483,  96]

In [ ]:
Seeds1 = Seeds2[1::2]

In [ ]:
def AddVarGen(df, parameters, PB, variability_types, replicate):
  """Adds given error types sequentially to given parameters.
  Number of Seeds required for RandomState generation = number of parameters*number of replicates.
  Seeds array should be created before calling function, and saved for validation of analysis."""

  """The norm.ppf function calculates is the inverse of the cumulative distribution function (CDF) of the standard normal distribution.
  ppf stands for percent point function, which is another name for the quantile function.
  random_sample selects a random number in the half-open interval [0,1). 1 Might occur due to rounding.
  Initially included 0 and 1 but this results in extreme values. Numpy rounds after 16 decimals so I used 15 decimals to create the included range, excluding 0 and 1
  providing a RandomState means the pseudo-random numbers generated will be the same each time the code is run. The number selected is meaningless. It just provides a starting point.
  The RandomState number should be changed 5 times to get 5 iterations of the error scenarios.
  Using (loc = mean, scale = SD) you can specify the mean and SD of the distribution instead of the default standard distribution."""

  
  newdf = df.copy()
  lp = len(parameters)
  lv = len(variability_types)

  for n,p in enumerate(parameters):
    if PB == 0 or p == 'dLDL':
      p2 = p
    else:
      p2 = f'{p}_PB{PB}'


    newdf.loc[:,p] = df_base.loc[:,p2]

    for k,e in enumerate(variability_types):
      pos = lp*lv*replicate + (lv*n +k)

      if lv == 1:
        a = np.random.RandomState(Seeds1[pos])
      else:
        a = np.random.RandomState(Seeds2[pos])
        
      if p == 'HDL' and e == 'CV':
        V = df_base.loc[:,f'HDL_CV{PB}']
      else:
        V = TAE.loc[p,e]


      newdf.loc[:,p] = norm.ppf(a.uniform(1e-16,0.9999999999999999,8506), loc = newdf.loc[:,p], scale = V*newdf.loc[:,p]).astype('int')

    #To deal with values <0, make them = 1
    newdf.loc[:,p] = np.select(condlist = [newdf.loc[:,p]<0, newdf.loc[:,p]>=0],
                               choicelist = [1, newdf.loc[:,p]],
                               default = np.nan)
  
  return newdf

## 2. Calculate and create columns for variable PCE terms and sum them

In [ ]:
def AddSum(newdf):
  """This function creates columns for the variable terms in the PCE calculation and sums them"""

  HDL = newdf['HDL']
  TC = newdf['TC']

  variableterms = ['TotalChol', 'AgeTotalChol','HDLChol', 'AgeHDLChol']
  variables = [np.log(TC), np.log(age)*np.log(TC), np.log(HDL), np.log(age)*np.log(HDL)]
  variablecoeffs = ['CTotalChol', 'CAgeTotalChol','CHDLChol', 'CAgeHDLChol']

  for t in range(4):
    newdf.loc[:,variableterms[t]] = variables[t]*df_base[variablecoeffs[t]]
  newdf.loc[:,'add_sum'] = newdf.loc[:,variableterms].sum(axis=1)
  
  return newdf

## 3. Pooled Cohort Equation (PCE)

In [ ]:
def PCEcalc(interum_sum, add_sum, meanterms, S10):
  terms = interum_sum + add_sum
  PCE = 100*(1 - S10 ** (np.e ** (terms - meanterms)))
  return PCE

## 4. Calculate and create LDL-C, non-HDL-C and PCE Columns

In [ ]:
def Calcs(df):
  """This function calculates LDL-C using the Friedewald equation, non-HDL-C,
  and the ASCVD risk using the pooled cohort equation (PCE) from the American Heart Association."""

  newdf = df.copy()
  HDL = newdf['HDL']
  TC = newdf['TC']
  TG = newdf['TG']
    
  newdf.loc[:,'NHDL'] = TC - HDL
  NHDL = newdf.loc[:,'NHDL']

  # To calculate LDL-C: where TG<=400, use Friedewald, where >400, use Sampson  
  newdf.loc[:,'cLDL'] = np.select(condlist = [newdf.loc[:,'TG']<=400, newdf.loc[:,'TG']>400],
                                  choicelist = [TC - HDL - TG/5, TC/0.948 - HDL/0.971 - (TG/8.56 + (TG*NHDL)/2140 - (TG**2)/16100) - 9.44],
                                  default = np.nan)  

  newdf.loc[:,'PCE'] = newdf.apply(lambda x: PCEcalc(x.interum_sum, x.add_sum, x.MeanTerms, x.S10), axis=1)

    
  return newdf

## 5. Risk Group Determination

In [ ]:
def Groups(df):
  """This function determines the Risk Group (A-D) based on LDL-C and non-HDL-C and creates a column for each."""

  newdf = df.copy()
  LDL = newdf['cLDL']
  NHDL = newdf['NHDL']
  PCE = newdf['PCE']
  TG = newdf['TG']

  newdf.loc[:,'Lgroup'] = np.nan
  newdf.loc[:,'Lgroup'] = np.select(condlist=[(PCE<7.5) | (LDL<70),
                                    (PCE >=20) | (LDL >=190),
                                    (LDL >=70) & (LDL <160) & (TG <175),
                                    (LDL >=160) | (TG >=175)],
                          choicelist=['A','D', 'B', 'C'],
                          default=np.nan)
    
  newdf.loc[:,'Ngroup'] = np.nan
  newdf.loc[:,'Ngroup'] = np.select(condlist=[(PCE<7.5) | (NHDL<90),
                                     (PCE >=20) | (NHDL >=220),
                                     (NHDL >=90) & (NHDL <190) & (TG <175),
                                     (NHDL >=190) | (TG >=175)],
                           choicelist=['A','D', 'B', 'C'],
                           default=np.nan)

  return newdf

##6. Call all functions to complete new databases

In [ ]:
def AddCols(df,parameters, PB, variability_types, replicate):
  """This function adds variability to the lipid results and then calculates the terms for PCE calculation that are dependent on these.
    It then sums those terms."""

#Create a Dataframe
  newdf = df[['ID','S10','MeanTerms','interum_sum']].copy()

#Add Variability and Sum of variable terms
  newdf = AddVarGen(newdf, parameters, PB, variability_types, replicate)
  newdf = AddSum(newdf)

#Calculate LDL-C, non-HDL-C and PCE and determine risk groups
  newdf = Calcs(newdf)
  newdf = Groups(newdf)

  return newdf

##7. Cross-tabulate new risk groups against original risk groups.

In [ ]:
def List_XT(dfList, group):
  """This function creates a list of dataframes created by cross-tabulation of the new risk group against the original risk group.
  Rows excluded from new dataframes due to lipid measurements <0 or TG >400, also excluded from df0 here for cross-tabulation"""

  XTList = []
  for df in dfList:
    XT = pd.DataFrame(pd.crosstab(df[group], df0[group], normalize = True))
    XTList.append(XT)
    
  return XTList

##8. Calculate the total reclassification across all groups for each cross-tabulation.

In [ ]:
def ReClass(XTList):
  """This function calculates the percentage of individuals that were reclassified due to the added error
  and makes a list of those percentages for each of the seven errors added.
  It also makes a list of the cross-tabulations."""

  TotalRC = []
  for df in XTList:
    total_reclassified = round(100*(1-df.loc['A','A']-df.loc['B','B']-df.loc['C','C']-df.loc['D','D']), ndigits=1)
    TotalRC.append(total_reclassified)

  return TotalRC

##9. Describe the distribution of reclassifications into each group given 50 (or any number of) replicates.

In [ ]:
def AveReclass(dfList):
  """This function concatenates all replicates of cross-tabulation and finds the means for each cell."""

  loXT_means = []
  for l in dfList:
    XT_concat = pd.concat(l)
    XT_group = XT_concat.groupby(XT_concat.index)
    XT_means_dict = XT_group.mean()
    XT_means = pd.DataFrame(XT_means_dict)
    loXT_means.append(XT_means)

  return loXT_means

# NEW LIPID RESULTS: STEPS 1-7

### Create global dataframes and lists for analysis

In [ ]:
dfnames = ('df1','df2','df3','df4','df5','df6','df7')
XTfilenames = ('XT1', 'XT2', 'XT3', 'XT4', 'XT5', 'XT6', 'XT7')

In [ ]:
steps = {'Step': np.arange(1,8), 'Criteria': 'new', 'Scenario': [0.0,0.1,0.2,1.1,1.2,2.1,2.2], 
           'Error Added': ['BV', 'CV', 'BV + CV', 'PB1 + CV', 'PB1 + BV + CV', 'PB2 + CV', 'PB2 + BV + CV']}
Error_L = pd.DataFrame(steps)
Error_N = pd.DataFrame(steps)

In [ ]:
L_List = [[], [], [], [], [], [], []]
N_List = [[], [], [], [], [], [], []]

# Perform analysis X times

(I have done X = 50. This involves generating 50 random numbers.)

Make directories

In [ ]:
for m in tqdm(range(50)):


#Create 7 new databases with different error added to lipid panel, and then calculate new risk scores and risk groups.

  df1 = AddCols(df_base, params, 0, ['BV'], m)
  df2 = AddCols(df_base, params, 0, ['CV'], m)
  df3 = AddCols(df_base, params, 0, ['BV', 'CV'], m)
  df4 = AddCols(df_base, params, 1, ['CV'], m)
  df5 = AddCols(df_base, params, 1, ['BV','CV'], m)
  df6 = AddCols(df_base, params, 2, ['CV'], m)
  df7 = AddCols(df_base, params, 2, ['BV','CV'], m)

  lodf = [df1, df2, df3, df4, df5, df6, df7]

#Create sets of 7 cross-tabulations between new and old risk groups based on LDL-C or non-HDL-C
  XTList_L = List_XT(lodf, 'Lgroup')
  XTList_N = List_XT(lodf, 'Ngroup')

#Create a dataframe showing the total reclassifications due to 7 error sub-scenarios, with a new column for each replicate analysis
  Error_L[f'ReClassd_{m+1}'] = ReClass(XTList_L)
  Error_N[f'ReClassd_{m+1}'] = ReClass(XTList_N)

#Create sets of 50 replicate across-tabulations for each of 7 error sub-scenarios, based on LDL-C or non-HDL-C
  for l,x in zip(L_List, XTList_L):
    l.append(x)

  for l,x in zip(N_List, XTList_N):
    l.append(x)
  
#Save non-global dataframes to .csv files
  n = m+1  

  for df,b in zip(lodf,dfnames):
    df.to_csv(f'New_Criteria/{b}/{b}_{n}.csv')
  
  for x,y,a in zip(XTList_L, XTList_N, XTfilenames):
    x.to_csv(f'New_Criteria/{a}/{a}L_{n}.csv')
    y.to_csv(f'New_Criteria/{a}/{a}N_{n}.csv')

100%|██████████| 50/50 [02:16<00:00,  2.73s/it]


# Save global dataframes to .CSV files

In [ ]:
Error_L['max'] = Error_L.iloc[:,4:].max(axis=1)
Error_L['min'] = Error_L.iloc[:,4:].min(axis=1)
Error_N['max'] = Error_N.iloc[:,4:].max(axis=1)
Error_N['min'] = Error_N.iloc[:,4:].min(axis=1)

In [ ]:
Error_L.to_csv('New_Criteria/Error/errorL.csv', index = False)
Error_N.to_csv('New_Criteria/Error/errorN.csv', index = False)

In [ ]:
loXT_means_L = AveReclass(L_List)
loXT_means_N = AveReclass(N_List)
for l,n,s in zip(loXT_means_L, loXT_means_N, range(1,8)):
  l.to_csv(f'New_Criteria/Stats/L{s}_means.csv')
  n.to_csv(f'New_Criteria/Stats/N{s}_means.csv')